In [76]:
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import time

## 1. Set-up urls to be scraped

In [2]:
# Set-up dictionary of all Mars urls to be scraped
url_dict = {"NASA":"https://mars.nasa.gov/news/",\
           "JPL":"https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars",\
           "Twitter":"https://twitter.com/marswxreport?lang=en",\
           "space-facts":"https://space-facts.com/mars/",\
           "geology":"https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"}

## 2. Scrape NASA and extract latest news _title_ and _text_

In [53]:
response = requests.get(url = url_dict["NASA"])

soup = bs(response.text, "html.parser")

# print(soup.prettify())

results = soup.find_all("div", class_ = "slide")

mars_latest_news = []

for result in results:    
    try:    
        news_title = result.find("div", class_ = "content_title").text
        news_description = result.find("div", class_ = "rollover_description_inner").text

        news_dict = {'title':news_title.strip(),\
                    'description':news_description.strip()}

        mars_latest_news.append(news_dict)
    except AttributeError as e:
        print(e)        

In [64]:
# Scanity-check
for news in mars_latest_news:
    print('Title: '+ news['title'])
    print('Descirption: '+news['description'])
    print("")

Title: Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
Descirption: NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries.

Title: NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network
Descirption: Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.

Title: NASA Administrator Statement on Moon to Mars Initiative, FY 2021 Budget
Descirption: Jim Bridenstine addresses NASA's ambitious plans for the coming years, including Mars Sample Return.

Title: NASA's Mars 2020 Rover Closer to Getting Its Name
Descirption: 155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.

Title: NASA Invites Students to Name Mars 2020 Rover
Descirption: Through Nov. 1, K-12 students in 

## 3. Scrape JPL and extract latest Mars featured image (using Splinter)

In [125]:
# Initiate the browser and visit JPL page

browser = Browser('chrome', headless=False)
browser.visit(url_dict['JPL'])

# Wait for 30 sec while the page loads
time.sleep(30)

# scrape the page
html = browser.html
soup = bs(html, 'html.parser')
featured_image = soup.find("li", class_="slide").a['data-fancybox-href']

# create full-url to high-res image
featured_image_url = url_dict['JPL'].split('/spaceimages')[0]+featured_image_url

In [126]:


# sanity-check
# browser.visit(featured_image_url)